In [1]:
import csv
import pandas as pd
import glob
import math
from pathlib import Path

In [106]:
hpc_codes_dir = "/Volumes/Huitian/Projects/T_Cell_ChIP/202012_ChIP/codes_hpc"

#--- Get list of sra files, retrive Run IDs
sra_dir = "/Volumes/Huitian/Projects/T_Cell_ChIP/202012_ChIP/1_SRA_Run_Table_simplified"
sra_files = glob.glob("%s/*.csv"%sra_dir)

srr_list = []
for file in sra_files:
    srr_list += pd.read_csv(file)['Run'].tolist()

In [103]:
#--- Write Hpc script
out_file = "2_0_MACS2"
out_dir = hpc_codes_dir + "/" + out_file
Path(out_dir).mkdir(parents=True, exist_ok=True)

hpc_wkdir = "/gpfs/group/pipkin/hdiao/T_Cell_ChIP/1_bowtie2"
hpc_outdir_1 = "/gpfs/group/pipkin/hdiao/T_Cell_ChIP/2_MACS2"
hpc_outdir_2 = "/gpfs/group/pipkin/hdiao/T_Cell_ChIP/2_MACS2_broad"

# Remove the ones that failed
failed_alignments = "/Volumes/Huitian/Projects/T_Cell_ChIP/202012_ChIP/codes_hpc/1_0_trim_alignment_flb_check_3/failed_alignments.csv"
failed_srr = pd.read_csv(failed_alignments)['Run'].tolist()
failed_srr

#--- Loop through all experiments
srr_used = []
for i in sra_files:
    
    i_df = pd.read_csv(i)
    i_name = i.split("/")[-1].replace("_simplified.csv","")
    
    testlist = i_df['condition'].tolist() + i_df['name'].tolist() + i_df['Input'].tolist()
    if "input" in "_".join([x for x in testlist if str(x) != "nan"]):
        print(i, "input error")
    
    i_df['sample_type'] = [x[:-2] for x in i_df['name']]
    i_sp_types = list(set(i_df['sample_type'].tolist()))
    i_sp_types = [x for x in i_sp_types if "Input" not in x]

    #####---------- Write one script for each sample condition
    # -t treatmentfile
    # -c control
    # -n name
    # -g genome
    # -B --SPMR generate pileup signal file of 'fragment 
    #            pileup per million reads' in bedGraph format.
    # --nomodel bypass building the shifting model
    # --extsize extend reads in 5'->3' direction to fix-sized 
    #           fragments when nomodel is on
    # --outdir save output fiels to specified folder
    # --broad  composite broad regions in BED12 (a gene-model-like 
    #          format) by putting nearby highly enriched regions 
    #          into a broad region with loose cutoff.

    for sp_type in i_sp_types:
        i_outname = "%s/%s_%s-%s.sh"%(out_dir, out_file, i_name, sp_type)

        sp_type_df = i_df[i_df['sample_type'] == sp_type]
        sp_srr = sp_type_df['Run'].tolist()
        sp_name = sp_type_df['condition'].tolist()[0]

        sp_input = sp_type_df['Input'].tolist()[0]
        if sp_input != "":
            sp_input_df = i_df[i_df['sample_type'] == sp_input]
            sp_input_srr = sp_input_df['Run'].tolist()
        else:
            sp_input_srr = []

        sp_srr = [x for x in sp_srr if x not in failed_srr]
        sp_input_srr = [x for x in sp_input_srr if x not in failed_srr]
        srr_used += sp_srr
        srr_used += sp_input_srr

        sp_bam_files = ["%s_srt_dupr_flb.bam"%x for x in sp_srr]
        sp_input_bam_files = ["%s_srt_dupr_flb.bam"%x for x in sp_input_srr]
        sp_bam_str = " ".join(sp_bam_files)
        sp_input_bam_str = " ".join(sp_input_bam_files)
        
        # Proceed if the experiment file is okay
        if len(sp_srr) > 0:
            with open(i_outname, "w") as fout:
                wfout = csv.writer(fout, delimiter="\t",lineterminator='\n')
                wfout.writerow(["#!/bin/bash"])
                wfout.writerow(["#SBATCH --mem=24gb"])
                wfout.writerow([])
                wfout.writerow(["module load macs/2.1.0"])
                wfout.writerow([])
                wfout.writerow(["BAM_INPUT_DIR=%s"%hpc_wkdir])
                wfout.writerow(["OUT_DIR1=%s"%hpc_outdir_1])
                wfout.writerow(["OUT_DIR2=%s"%hpc_outdir_2])
                wfout.writerow(["cd $BAM_INPUT_DIR"])
                wfout.writerow([])
                wfout.writerow(["########## Peak calling"])
                if len(sp_input_srr) > 0:
                    wfout.writerow(["macs2 callpeak -t %s -c %s \\" 
                                    %(sp_bam_str, sp_input_bam_str)])
                else:
                    wfout.writerow(["macs2 callpeak -t %s \\"
                                    %(sp_bam_str)])
                wfout.writerow(["               -g mm -B --SPMR --nomodel --extsize 147 --call-summits \\"])
                wfout.writerow(["               -n %s___%s --outdir $OUT_DIR1"%(i_name, sp_name)])

                if sp_type.startswith("H3"):
                    wfout.writerow([])
                    if len(sp_input_srr) > 0:
                        wfout.writerow(["macs2 callpeak -t %s -c %s \\" 
                                        %(sp_bam_str, sp_input_bam_str)])
                    else:
                        wfout.writerow(["macs2 callpeak -t %s \\"
                                        %(sp_bam_str)])
                    wfout.writerow(["               -g mm --nomodel --extsize 147 --broad \\"])
                    wfout.writerow(["               -n %s___%s --outdir $OUT_DIR2"%(i_name, sp_name)])

In [105]:
len(set(srr_used))

177

In [107]:
len(srr_list)

181